In [29]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

Файл disambiguated_corpus содержит тексты на кыргызском языке, размеченные программой Apertium (морфологическая разметка). Для каждого слова есть от одного (и больше) варианта морфологического разбора. Также проведенна ручная POS разметка (частеречная).

Задача: снять частеречную омонимию, т.е. каждому слову проставить частеречный тег

In [7]:
import json

with open('disambiguated_corpus.json', 'r') as fp:
    row_data = json.load(fp)

In [8]:
# всего слов
len(row_data)

20040

In [19]:
# пример вариантов морфологического разбора для слова "Саякбай"

row_data[2]

# здесь 'np' - тег слова, в нашем случае таргет 
# word - само слово, 
# lemmas - список вариантов начальных форм слова
# lemma_var - начальная форма слова
# tags - грамматические теги

['np',
 {'word': 'Саякбай',
  'lemmas': [{'lemma_var': 'Саякбай', 'tags': ['np', 'ant', 'm', 'nom']},
   {'lemma_var': 'Саякбай',
    'tags': ['np', 'ant', 'm', 'nom', '+э<cop', 'aor', 'p3', 'pl']},
   {'lemma_var': 'Саякбай',
    'tags': ['np', 'ant', 'm', 'nom', '+э<cop', 'aor', 'p3', 'sg']}]}]

In [38]:
# генерирую дополнительные фичи для каждого слова из имеющихся данных

def get_features(word_data, additional_info = ''):
    
    POS_unique = True
    features = {}

    POS_variantes = []
    lemmas_unique = []

    for lemma in word_data['lemmas']:

        if len(lemma['tags']) > 0:

            if lemma['lemma_var'] not in lemmas_unique:

                if lemma['tags'][0] not in POS_variantes:

                    POS_variantes.append(lemma['tags'][0])
                    lemmas_unique.append(lemma['lemma_var'])

#     POS_feat = {f'{i}_{additional_info}_pos': key for i, key in enumerate(POS_variantes)}
    lemmas_feat = {f'{i}_{additional_info}_lemma': key for i, key in enumerate(lemmas_unique)}
#     features.update(POS_feat)
    features.update(lemmas_feat)

    features.update({f'{additional_info}_word.lower()': word_data['word'].lower(),
            f'{additional_info}_word[-3:]': word_data['word'][-3:],
            f'{additional_info}_word[-2:]': word_data['word'][-2:],
            f'{additional_info}_word.isupper()': word_data['word'].isupper(),
            f'{additional_info}_word.istitle()': word_data['word'].istitle(),
            f'{additional_info}_word.isdigit()': word_data['word'].isdigit()})
#             'context_forward' : word_data[ind_word+1],
#             'context_backward': word_data[ind_word-1]})
    return features


def run_fest(corpus):
    train_data = []
    print
    for ind_word, (y, word) in enumerate(corpus):

        word_features = get_features(word)
        word_features.update({'y': y.split('>')[0], 
                             'word': word['word']})

        if ind_word == 0:
            
            word_features.update(get_features(corpus[ind_word+1][1], 'forw'))

        elif ind_word == len(corpus) -1 :
            word_features.update(get_features(corpus[ind_word-1][1], 'back'))

        else:
#             print(ind_word)
            word_features.update(get_features(corpus[ind_word+1][1], 'forw'))
            word_features.update(get_features(corpus[ind_word-1][1], 'back'))
        
        
        train_data.append(word_features)
    return train_data

In [39]:
# у каждого слова есть фичи о самом слове 
# и об окужающем контексте (соседних словах)

data = run_fest(itog)
data[3]

{'0__lemma': 'Каралаев',
 '_word.lower()': 'каралаевдин',
 '_word[-3:]': 'дин',
 '_word[-2:]': 'ин',
 '_word.isupper()': False,
 '_word.istitle()': True,
 '_word.isdigit()': False,
 'y': 'np',
 'word': 'Каралаевдин',
 '0_forw_lemma': 'айтым',
 'forw_word.lower()': 'айтымында',
 'forw_word[-3:]': 'нда',
 'forw_word[-2:]': 'да',
 'forw_word.isupper()': False,
 'forw_word.istitle()': False,
 'forw_word.isdigit()': False,
 '0_back_lemma': 'Саякбай',
 'back_word.lower()': 'саякбай',
 'back_word[-3:]': 'бай',
 'back_word[-2:]': 'ай',
 'back_word.isupper()': False,
 'back_word.istitle()': True,
 'back_word.isdigit()': False}

In [40]:
# разбиваю данные на предложения

def data2sent(i):
    sentence = []
    for x in data[i:]:
#         print(i, x['y'])
        if x['y'] == 'sent':
            
            sentence.append(x)
            i += 1
#             print(i, x['y'])
#             print(type(sentence), i)
            return sentence, i
        else:
            sentence.append(x)
            i += 1
    return sentence, i


i = 0
sentences = []
while i < 20040:
    if data2sent(i) == None:
        i = 20040
    else:
        s, i = data2sent(i)
    
        sentences.append(s)

In [41]:
# перемешиваю предложения перед разбиением на трейн и тест
import random
random.shuffle(sentences)

sentences[1] 

[{'0__lemma': 'ошол',
  '_word.lower()': 'ошо',
  '_word[-3:]': 'Ошо',
  '_word[-2:]': 'шо',
  '_word.isupper()': False,
  '_word.istitle()': True,
  '_word.isdigit()': False,
  'y': 'det',
  'word': 'Ошо',
  '0_forw_lemma': 'Жакып',
  'forw_word.lower()': 'жакып',
  'forw_word[-3:]': 'кып',
  'forw_word[-2:]': 'ып',
  'forw_word.isupper()': False,
  'forw_word.istitle()': True,
  'forw_word.isdigit()': False,
  '0_back_lemma': '.',
  'back_word.lower()': '.',
  'back_word[-3:]': '.',
  'back_word[-2:]': '.',
  'back_word.isupper()': False,
  'back_word.istitle()': False,
  'back_word.isdigit()': False},
 {'0__lemma': 'Жакып',
  '_word.lower()': 'жакып',
  '_word[-3:]': 'кып',
  '_word[-2:]': 'ып',
  '_word.isupper()': False,
  '_word.istitle()': True,
  '_word.isdigit()': False,
  'y': 'np',
  'word': 'Жакып',
  '0_forw_lemma': 'кайран',
  'forw_word.lower()': 'кайран',
  'forw_word[-3:]': 'ран',
  'forw_word[-2:]': 'ан',
  'forw_word.isupper()': False,
  'forw_word.istitle()': False,

In [42]:
X_train = []
y_train = []
for sent in sentences[round(len(sentences)/100*20):]:
    y_train.append([x.pop('y') for x in sent])
    X_train.append([x for x in sent])

    
X_test = []
y_test = []
for sent in sentences[:round(len(sentences)/100*20)]:
    y_test.append([x.pop('y') for x in sent])
    X_test.append([x for x in sent])


In [43]:
# пример таргетов для первых двух предложений
y_train[:2]

[['n',
  'np',
  'cm',
  'lquot',
  'rquot',
  'np',
  'num',
  'num',
  'num',
  'lpar',
  'num',
  'rpar',
  'abbr',
  'num',
  'n',
  'sent'],
 ['v', 'v', 'cm', 'n', 'adj', 'postadv', 'prn', 'cop', 'sent']]

In [44]:
# пример предикторов для первого предложения

X_train[0]

[{'0__lemma': 'Айнек',
  '1__lemma': 'айнек',
  '_word.lower()': 'айнек',
  '_word[-3:]': 'НЕК',
  '_word[-2:]': 'ЕК',
  '_word.isupper()': True,
  '_word.istitle()': False,
  '_word.isdigit()': False,
  'word': 'АЙНЕК',
  '0_forw_lemma': 'Жайнаков',
  'forw_word.lower()': 'жайнакова',
  'forw_word[-3:]': 'ОВА',
  'forw_word[-2:]': 'ВА',
  'forw_word.isupper()': True,
  'forw_word.istitle()': False,
  'forw_word.isdigit()': False,
  '0_back_lemma': ':',
  'back_word.lower()': ':',
  'back_word[-3:]': ':',
  'back_word[-2:]': ':',
  'back_word.isupper()': False,
  'back_word.istitle()': False,
  'back_word.isdigit()': False},
 {'0__lemma': 'Жайнаков',
  '_word.lower()': 'жайнакова',
  '_word[-3:]': 'ОВА',
  '_word[-2:]': 'ВА',
  '_word.isupper()': True,
  '_word.istitle()': False,
  '_word.isdigit()': False,
  'word': 'ЖАЙНАКОВА',
  '0_forw_lemma': ',',
  'forw_word.lower()': ',',
  'forw_word[-3:]': ',',
  'forw_word[-2:]': ',',
  'forw_word.isupper()': False,
  'forw_word.istitle()': 

In [45]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
    verbose=True
)
try:
    crf.fit(X_train, y_train)
    
except AttributeError:
    pass
y_pred = crf.predict(X_test)

loading training data to CRFsuite: 100%|███████████████████████████████████████████| 870/870 [00:00<00:00, 3537.68it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 51868
Seconds required: 0.099

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.07  loss=44784.29 active=51262 feature_norm=1.00
Iter 2   time=0.04  loss=40841.89 active=51235 feature_norm=12.59
Iter 3   time=0.04  loss=24949.98 active=49312 feature_norm=11.86
Iter 4   time=0.04  loss=23462.40 active=50531 feature_norm=11.68
Iter 5   time=0.04  loss=20533.87 active=51203 feature_norm=11.77
Iter 6   time=0.04  loss=18624.60 active=50792 feature_norm=12.61
Iter 7   time=0.04  loss=16494.96 active=50924 feature_norm=14.07
Iter 8   time=0.04  loss=13153.65 active=50021 feature_norm=18.68
Iter 9   time=0.04  loss=10649.81 active=48738 feature_norm=23.53
Iter 1

In [46]:
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', zero_division=0)

0.9483813356940394

In [47]:
# количество классов 

len(crf.classes_)

25

In [48]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer

m = MultiLabelBinarizer().fit(y_test)

print("F1-score is : {:.1%}".format(f1_score(m.transform(y_test),
         m.transform(y_pred),
         average='macro')))
print(classification_report(m.transform(y_test), m.transform(y_pred), target_names=m.classes_))

F1-score is : 94.1%
              precision    recall  f1-score   support

         adj       0.96      0.94      0.95       139
         adv       0.97      0.86      0.91        43
          cm       1.00      1.00      1.00       178
      cnjadv       1.00      0.33      0.50         3
      cnjcoo       1.00      0.91      0.96        47
         cop       1.00      0.96      0.98        26
         det       1.00      0.90      0.95        42
        guio       1.00      1.00      1.00        34
          ij       0.93      0.87      0.90        15
        lpar       1.00      1.00      1.00         6
       lquot       1.00      1.00      1.00        28
           n       0.99      1.00      0.99       196
          np       1.00      0.93      0.96        55
         num       0.98      0.94      0.96        64
        post       1.00      0.83      0.91         6
     postadv       0.89      1.00      0.94         8
         prn       1.00      0.96      0.98        85
       

довольно неплохая точность с первого приближения, дальнейшие шаги: 
- применение других моделей
- очистка данных
- балансировка по классам
- увеличение датасета